In [1]:
%cd drive/MyDrive/LLMs

/content/drive/MyDrive/LLMs


# Dependencies

In [103]:
!pip install datasets --quiet
!pip install textstat --quiet
!pip install wandb --quiet
!pip install redditcleaner --quiet
!pip install huggingface_hub --quiet
!pip install -U sentence-transformers --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.7/214.7 kB 19.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.8 MB/s eta 0:00:00


In [184]:
import wandb, torch
import sys
import os
import redditcleaner
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from huggingface_hub import notebook_login
from sentence_transformers import SentenceTransformer
from textstat import flesch_reading_ease as fre
from datasets import load_dataset, load, load_from_disk
from itertools import compress

device = "cuda" if torch.cuda.is_available() else "cpu"

%matplotlib inline

In [105]:
notebook_login()

#creates the 'results' and 'data' directories, if they don't exist.
if not os.path.exists('results'):
    os.makedirs('results')

if not os.path.exists('data'):
    os.makedirs('data')

# Definitions

In [196]:
def compute_FRE(example):
    fre_scores = [fre(text) for text in example['answers']['text']]
    example['answers']['fre'] = fre_scores
    return example

def preprocess_example(example):

    answers = example['answers']['text']
    answers = [redditcleaner.clean(answer) for answer in answers]
    answers = [re.sub('>.*?\n',' ',answer) for answer in answers]
    answers = [' '.join(answer.lower().split()) for answer in answers]
    example['answers']['text'] = answers

    title = example['title']
    title = redditcleaner.clean(title)
    title = ' '.join(title.split())
    example['title'] = title

    selftext = example['selftext']
    selftext = redditcleaner.clean(selftext)
    selftext = ' '.join(selftext.lower().split())
    example['selftext'] = selftext

    return example

def preprocess_data(dataset):
    dataset = dataset.map(preprocess_example)
    return dataset

class score_cutoff_wrapper:
    def __init__(self,cutoff):
        self.cutoff = cutoff

    def score_cutoff_ex(self,example):
        scores = example['answers']['score']
        idxs = list(np.array(scores) >= self.cutoff)
        for key, val in example['answers'].items():
            example['answers'][key] = list(compress(val,idxs))

        return example

def score_cutoff(dataset,cutoff):
    cutoff = score_cutoff_wrapper(cutoff)
    ds = dataset.map(cutoff.score_cutoff_ex)
    ds = ds.filter(lambda post: len(post['answers']['score']>0))

    return ds

# Filtering Data

In [94]:
dataset = load_dataset("vblagoje/lfqa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [163]:
dataset_preprocessed = preprocess_data(dataset)

Map:   0%|          | 0/226147 [00:00<?, ? examples/s]

Map:   0%|          | 0/3020 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [164]:
dataset_preprocessed.save_to_disk('./data/preprocessing')

Saving the dataset (0/2 shards):   0%|          | 0/226147 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3020 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
not_qus = ['IAMA','AMA','ama:','megathread','Megathread',
           'Discussion Thread','Discussion thread',
           'discussion Thread','discussion thread']

qu_reqs = ['who','what','where','why','when','how','?']

In [169]:
ds_reduced_pt1 = dataset_preprocessed.filter(lambda post:
                                         not (all(qu_req not in post['title'].lower() for qu_req in qu_reqs)
                                         and all(qu_req not in post['selftext'].lower() for qu_req in qu_reqs)))

Filter:   0%|          | 0/226147 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3020 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [170]:
ds_reduced_pt2 = ds_reduced_pt1.filter(lambda post:
                                       not (any(nq in post['title'] for nq in not_qus)))

Filter:   0%|          | 0/221802 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2983 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9734 [00:00<?, ? examples/s]

In [173]:
ds_mult_ans = ds_reduced_pt2.filter(lambda post : len(post['answers']['score'])>=2)
ds_sing_ans = ds_reduced_pt2.filter(lambda post : len(post['answers']['score'])==1)

Filter:   0%|          | 0/221585 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2973 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9707 [00:00<?, ? examples/s]

Filter:   0%|          | 0/221585 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2973 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9707 [00:00<?, ? examples/s]

In [ ]:
ds_mult_ans_cutoff = score_cutoff(ds_mult_ans,4)

Map:   0%|          | 0/127102 [00:00<?, ? examples/s]

In [194]:
ds_mult_ans[]

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 127102
    })
    validation: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 2302
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 6498
    })
})

In [193]:
ds_mult_ans_cutoff

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 127102
    })
    validation: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 2302
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 6498
    })
})

# Scratch

In [ ]:
ds_eli5_pref = ds_eli5.filter(lambda example: len(example['answers']['score'])>=2)
ds_eli5_solo = ds_eli5.filter(lambda example: len(example['answers']['score'])==1)

In [ ]:
def split_idxs(example):
    scores = example['answers']['score']
    scores_unique = sorted(set(scores),reverse=True)
    pref_scores_idxs = [scores.index(sc) for sc in scores_unique]
    dupl_scores_idxs = [n for n in range(len(scores)) if n not in pref_scores_idxs]

    example["num_ans"] = {'pref_idxs' : pref_scores_idxs,
                          'dupl_idxs' : dupl_scores_idxs}
    return example

In [ ]:
ds_eli5_pref_v2 = ds_eli5_pref.map(split_idxs)

In [ ]:
ds_eli5_flatten.filter(lambda x: np.max(x['answers.score'])>=5)

In [ ]:
ds_eli5_solo['train'].filter(lambda x : x['answers']['score'][0]>3)

Filter:   0%|          | 0/38019 [00:00<?, ? examples/s]

Dataset({
    features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
    num_rows: 15253
})

In [ ]:
!pip install py-readability-metrics
!python -m nltk.downloader punkt

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from readability import Readability

In [ ]:
text = ds_eli5_pref_v2['train'][4]['answers']['text'][0]

In [ ]:
r = Readability(text)

In [ ]:
r.flesch_kincaid().grade_level

'8'

In [ ]:
r.flesch().grade_levels

['8', '9']

In [ ]:
text=' '.join(dataset['train'][0]['answers']['text'][0].split())

In [ ]:
len(dataset['train'][0]['answers']['text'][0].split())

93

In [ ]:
list(map(lambda x:len(x.split()),ds_askh['train'][1]['answers']['text']))

[81, 396, 199]

In [ ]:
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.7 MB/s eta 0:00:00


In [ ]:
import textstat


In [ ]:
test_data = ds_eli5['train'][0]['answers']['text'][0]

print(textstat.flesch_kincaid_grade(test_data))
textstat.flesch_reading_ease(test_data)

9.2


61.46

In [ ]:
len(test_data.split())

3

In [ ]:
ds_flatten = dataset.flatten()

In [ ]:
ds_flatten

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers.a_id', 'answers.score', 'answers.text', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 226147
    })
    validation: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers.a_id', 'answers.score', 'answers.text', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 3020
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'url', 'answers.a_id', 'answers.score', 'answers.text', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 10000
    })
})

In [ ]:
df_flatten = pd.DataFrame(ds_flatten['train'])

In [ ]:
df_exploded = df_flatten.explode('answers.score')

In [ ]:
df_exploded

,q_id,title,selftext,document,subreddit,url,answers.a_id,answers.score,answers.text,title_urls,selftext_urls,answers_urls
0,32wvn8,what's the difference between a forest and a w...,,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,[cqfd1d8],2,[They're used interchangeably a lot. You'll ge...,[],[],[[]]
1,1yc9zg,Are there any good source material on the Wars...,"Hi guys, I have a project I'm working on which...",,AskHistorians,http://www.reddit.com/r/AskHistorians/comments...,[cfj9u9t],3,[Many of the relevant primary sources wont con...,[],[],[[http://www.worldcat.org/title/warsaw-diary-o...
2,elzx1n,we do we instinctively grab a part of our body...,I just tweaked my wrist and my immediate react...,,explainlikeimfive,https://www.reddit.com/r/explainlikeimfive/com...,"[fdlfvnn, fdltn5j]",11,[A) instinct. To protect it from further damag...,[],[],"[[], []]"
2,elzx1n,we do we instinctively grab a part of our body...,I just tweaked my wrist and my immediate react...,,explainlikeimfive,https://www.reddit.com/r/explainlikeimfive/com...,"[fdlfvnn, fdltn5j]",3,[A) instinct. To protect it from further damag...,[],[],"[[], []]"
3,1j7pwx,Following the passing of the Thirteenth Amendm...,I'm curious as to whether there were any remot...,,AskHistorians,http://www.reddit.com/r/AskHistorians/comments...,"[cbc0e13, cbc0gmv, cbc2ka2]",31,[All across the South during the years followi...,[],[],"[[], [S.D.Ga, http://books.google.com/books/ab..."
...,...,...,...,...,...,...,...,...,...,...,...,...
226145,1878a3,why do we refer to animals' colors not as the ...,I guess this is true with people too. But what...,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[c8c954h, c8c988t, c8ca8mq]",7,[I have absolutely no clue what you are talkin...,[],[],"[[], [], []]"
226145,1878a3,why do we refer to animals' colors not as the ...,I guess this is true with people too. But what...,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[c8c954h, c8c988t, c8ca8mq]",3,[I have absolutely no clue what you are talkin...,[],[],"[[], [], []]"
226145,1878a3,why do we refer to animals' colors not as the ...,I guess this is true with people too. But what...,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[c8c954h, c8c988t, c8ca8mq]",3,[I have absolutely no clue what you are talkin...,[],[],"[[], [], []]"
226146,1ntyey,why does climate change mean that climates wil...,I've always been curious as to why it is const...,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[ccm0sa1, ccm2lp3]",3,[Some areas may become less extreme. But in an...,[],[],"[[], []]"


In [ ]:
df_flatten

,q_id,title,selftext,document,subreddit,url,answers.a_id,answers.score,answers.text,title_urls,selftext_urls,answers_urls
0,32wvn8,what's the difference between a forest and a w...,,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,[cqfd1d8],[2],[They're used interchangeably a lot. You'll ge...,[],[],[[]]
1,1yc9zg,Are there any good source material on the Wars...,"Hi guys, I have a project I'm working on which...",,AskHistorians,http://www.reddit.com/r/AskHistorians/comments...,[cfj9u9t],[3],[Many of the relevant primary sources wont con...,[],[],[[http://www.worldcat.org/title/warsaw-diary-o...
2,elzx1n,we do we instinctively grab a part of our body...,I just tweaked my wrist and my immediate react...,,explainlikeimfive,https://www.reddit.com/r/explainlikeimfive/com...,"[fdlfvnn, fdltn5j]","[11, 3]",[A) instinct. To protect it from further damag...,[],[],"[[], []]"
3,1j7pwx,Following the passing of the Thirteenth Amendm...,I'm curious as to whether there were any remot...,,AskHistorians,http://www.reddit.com/r/AskHistorians/comments...,"[cbc0e13, cbc0gmv, cbc2ka2]","[31, 58, 10]",[All across the South during the years followi...,[],[],"[[], [S.D.Ga, http://books.google.com/books/ab..."
4,3qr7uu,"In medieval and pre-modern times, political en...",this peace didn't last*...,,AskHistorians,https://www.reddit.com/r/AskHistorians/comment...,[cwi14dl],[11],[Twenty years of peace is much better than no ...,[],[],[[https://en.wikipedia.org/wiki/Kingdom_of_Cas...
...,...,...,...,...,...,...,...,...,...,...,...,...
226142,2cy88t,why do dogs love peanut butter so much?,,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[cjk80e7, cjk80oo, cjk96qd, cjkkawu]","[6, 3, 2, 2]","[Sweet, salty. Why else?, I think it's more t...",[],[],"[[], [], [], []]"
226143,1vn3rg,why do companies trademark simple words that o...,"I was at Buffalo Wild Wings and the ""Mild"" sau...",,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,[cetwoz3],[3],"[A trademark is more than just the words, it i...",[],[],[[]]
226144,alst20,difference between hypoxemia and hypoxia,[deleted],,explainlikeimfive,https://www.reddit.com/r/explainlikeimfive/com...,"[efgnytp, efhhfjl]","[8, 4]",[Hypoxemia is having low oxygen content in you...,[],[],"[[], []]"
226145,1878a3,why do we refer to animals' colors not as the ...,I guess this is true with people too. But what...,,explainlikeimfive,http://www.reddit.com/r/explainlikeimfive/comm...,"[c8c954h, c8c988t, c8ca8mq]","[7, 3, 3]",[I have absolutely no clue what you are talkin...,[],[],"[[], [], []]"
